<a href="https://colab.research.google.com/github/divyyeahhhhh/Comparative_Analysis_Vitiligo_Progression/blob/main/ANN_for_vitiligo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from google.colab import drive
data_dir = '/content/drive/MyDrive/TEST DATASET'
def load_data(data_dir):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        for image in os.listdir(os.path.join(data_dir, label)):
            img = cv2.imread(os.path.join(data_dir, label, image))
            img = cv2.resize(img, (30, 30))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

images, labels = load_data(data_dir)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.08, random_state=42)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='sigmoid', input_shape=(30, 30, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='sigmoid'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='sigmoid'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=19), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)
accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f',accuracy)
# Retrieve the accuracy values from the training history
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
# Retrieve the accuracy values from the training history
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot the accuracy values as lines
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), train_accuracy, label='Training Accuracy')
plt.plot(range(1, 11), val_accuracy, label='Validation Accuracy')
plt.title('Accuracy per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

confusion = confusion_matrix(y_test, y_pred_binary)
print("Confusion Matrix:")
print(confusion)
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='magma', xticklabels=['Healthy Skin', 'Diseased Skin'], yticklabels=['Healthy Skin', 'Diseased Skin'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix Heatmap')
plt.show()

